In [2]:
import pandas as pd
import numpy as np

In [3]:
df_address = pd.read_csv('address(in).csv')
df_main = pd.read_csv('main(in).csv')
df_paid = pd.read_csv('paid_record(in).csv')

In [4]:
df_paid.head(20)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
5,5001712,-1,C
6,5001712,-2,C
7,5001712,-3,C
8,5001712,-4,C
9,5001712,-5,C


* 'X': No credit for the month.
* 'C': The credit is closed.
* '0': No overdue payments.
* Other numeric values represent the number of months the payment is overdue (e.g., '1', '2', etc.).

In [5]:
df_paid['STATUS'].unique()

array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object)

In [6]:
df_paid = df_paid.rename(columns={'ID': 'Main_ID'})
df_paid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   Main_ID         1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [7]:
df_main.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,DAYS_BIRTH_CLEAN,Letter,Main_ID
0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,L,5008804
1,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,G,5008805
2,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,06/05/1964,D,5008806
3,F,N,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,I,5008808
4,F,NaN,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,Z,5008809


In [8]:
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   New_ID   438757 non-null  object 
 1   ADDRESS  0 non-null       float64
dtypes: float64(1), object(1)
memory usage: 6.7+ MB


In [9]:
df_address = df_address.drop('ADDRESS', axis=1)
df_address[['id', 'address']] = df_address['New_ID'].str.split(',', n=1, expand=True)
df_address = df_address.drop('New_ID', axis=1)
df_address['Main_ID'] = df_address['id'].str.extract(r'(\d+)')
df_address = df_address.drop('id', axis=1)
df_address['Main_ID'] = df_address['Main_ID'].astype('Int64')
df = pd.merge(df_main, df_address, on='Main_ID', how='inner')
df = df.dropna(how='all')
df['state'] = df['address'].str.extract(r'([A-Z]{2})')
df['CNT_CHILDREN'] = df['CNT_CHILDREN'].replace('.','')
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace('.','')
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].abs()
df['AMT_INCOME_TOTAL'] = df['AMT_INCOME_TOTAL'].replace('.',',')
#df_paid_2 = df_paid.groupby('ID')['STATUS'].value_counts().unstack().reset_index().fillna(0)
#df_paid_2.rename(columns={'ID': 'Main_ID'}, inplace=True)
#df = pd.merge(df_main, df_paid_2, on='Main_ID', how='inner')

COLUMNS = ['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']
for col in COLUMNS:
    if df[col].mean() > 0:
        df[col] = df[col].replace({np.nan: df[col].mean()})
    else :   
        df[col] = df[col].replace(np.nan, 0)

COLUMNS = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE','FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL']
for col in COLUMNS:
    df[col] =  df[col].replace({np.nan: df[col].mode()[0]})

df['NAME_INCOME_TYPE'] = df['NAME_INCOME_TYPE'].fillna('SinInfo')
df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].fillna('other')
df['OCCUPATION_TYPE'] = df['OCCUPATION_TYPE'].fillna('other')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['AMT_INCOME_TOTAL'] = scaler.fit_transform(df[['AMT_INCOME_TOTAL']])
df['DAYS_EMPLOYED'] = scaler.fit_transform(df[['DAYS_EMPLOYED']])

df = df.merge(df_paid, how='inner', left_on='Main_ID', right_on='Main_ID')



In [10]:
df.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,...,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,DAYS_BIRTH_CLEAN,Letter,Main_ID,address,state,MONTHS_BALANCE,STATUS
0,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,...,0.0,other,2.0,09/04/1990,L,5008804,"""06042, CT, 31 Mitchell Road""",CT,0,C
1,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,...,0.0,other,2.0,09/04/1990,L,5008804,"""06042, CT, 31 Mitchell Road""",CT,-1,C
2,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,...,0.0,other,2.0,09/04/1990,L,5008804,"""06042, CT, 31 Mitchell Road""",CT,-2,C
3,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,...,0.0,other,2.0,09/04/1990,L,5008804,"""06042, CT, 31 Mitchell Road""",CT,-3,C
4,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,...,0.0,other,2.0,09/04/1990,L,5008804,"""06042, CT, 31 Mitchell Road""",CT,-4,C


In [11]:
df['DAYS_BIRTH_CLEAN'] = pd.to_datetime(df['DAYS_BIRTH_CLEAN'], format='%d/%m/%Y')
df['DAYS_BIRTH_CLEAN'] = df['DAYS_BIRTH_CLEAN'].fillna(df['DAYS_BIRTH_CLEAN'].mean())
df['ano'] = df['DAYS_BIRTH_CLEAN'].dt.year
df = df.drop('DAYS_BIRTH_CLEAN', axis=1)

In [12]:
df.isnull().sum() * 100 / len(df)

CODE_GENDER            0.0
FLAG_OWN_CAR           0.0
FLAG_OWN_REALTY        0.0
CNT_CHILDREN           0.0
AMT_INCOME_TOTAL       0.0
NAME_INCOME_TYPE       0.0
NAME_EDUCATION_TYPE    0.0
NAME_FAMILY_STATUS     0.0
NAME_HOUSING_TYPE      0.0
DAYS_EMPLOYED          0.0
FLAG_MOBIL             0.0
FLAG_WORK_PHONE        0.0
FLAG_PHONE             0.0
FLAG_EMAIL             0.0
OCCUPATION_TYPE        0.0
CNT_FAM_MEMBERS        0.0
Letter                 0.0
Main_ID                0.0
address                0.0
state                  0.0
MONTHS_BALANCE         0.0
STATUS                 0.0
ano                    0.0
dtype: float64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781085 entries, 0 to 781084
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   CODE_GENDER          781085 non-null  object 
 1   FLAG_OWN_CAR         781085 non-null  object 
 2   FLAG_OWN_REALTY      781085 non-null  object 
 3   CNT_CHILDREN         781085 non-null  float64
 4   AMT_INCOME_TOTAL     781085 non-null  float64
 5   NAME_INCOME_TYPE     781085 non-null  object 
 6   NAME_EDUCATION_TYPE  781085 non-null  object 
 7   NAME_FAMILY_STATUS   781085 non-null  object 
 8   NAME_HOUSING_TYPE    781085 non-null  object 
 9   DAYS_EMPLOYED        781085 non-null  float64
 10  FLAG_MOBIL           781085 non-null  float64
 11  FLAG_WORK_PHONE      781085 non-null  float64
 12  FLAG_PHONE           781085 non-null  float64
 13  FLAG_EMAIL           781085 non-null  float64
 14  OCCUPATION_TYPE      781085 non-null  object 
 15  CNT_FAM_MEMBERS  

In [15]:
df_use = df.drop(columns=['Main_ID', 'address', 'Letter'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

for column in df_use.columns:
    if df_use[column].dtype == type(object):
        le = LabelEncoder()
        df_use[column] = le.fit_transform(df_use[column])



In [16]:
df_use.head()   

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,state,MONTHS_BALANCE,STATUS,ano
0,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,1.0,1.0,0.0,0.0,other,2.0,CT,0,C,1990
1,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,1.0,1.0,0.0,0.0,other,2.0,CT,-1,C,1990
2,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,1.0,1.0,0.0,0.0,other,2.0,CT,-2,C,1990
3,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,1.0,1.0,0.0,0.0,other,2.0,CT,-3,C,1990
4,M,Y,Y,0.0,2.205991,Working,Higher education,Civil marriage,Rented apartment,-0.446581,1.0,1.0,0.0,0.0,other,2.0,CT,-4,C,1990


In [ ]:
data = df_use.sample(frac=0.8, random_state=342)
data

In [ ]:
data_unseen = df_use.drop(data.index)
data_unseen

In [ ]:
data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
from pycaret.regression import *
model_setup = setup(data = data, target = 'state', session_id=456)

In [ ]:
models()

In [ ]:
best_model = compare_models()

In [ ]:
model_rf = create_model('rf')
print(model_rf)

In [ ]:
evaluate_model(model_rf)

In [ ]:
plot_model(model_rf, plot = 'feature')

In [ ]:
data_unseen

In [ ]:
unseen_predictions = predict_model(model_rf, data=data_unseen)
unseen_predictions.head()

In [ ]:
final_Model = finalize_model(model_rf)

In [ ]:
save_model(final_Model, 'Final_predict_Model')

Creando el modelo final

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df_use.drop('STATUS', axis=1)
y = df_use['STATUS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=242)

RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)
y_preds = RFC.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds))

In [ ]:
flor = np.array([[5,3.2,1.3,0.15]])
RFC.predict(flor)